In [1]:
import os
import numpy as np
import tensorflow as tf
import json
import random
import copy

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt

data_folder = 'data_train'

In [2]:
with open('database_train.json') as json_file:
    y = json.load(json_file)

In [ ]:
%%time
x_train = []
y_train = []

for file_name in os.listdir(data_folder):
    img_path = os.path.join(data_folder, file_name)
    img = image.load_img(img_path, target_size=(32, 32))
    x = image.img_to_array(img)
    x = x/255
    
    """
    plt.figure(figsize=(3,3))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x, cmap=plt.cm.binary)
    plt.show()
    """
    
    x_train += [x]
    y_train += [np.array(y[file_name])]
    
x_valid = np.array(x_train)[:1000]
y_valid = np.array(y_train)[:1000]

x_train = np.array(x_train)[1000:]
y_train = np.array(y_train)[1000:]

In [ ]:
%%time
x_train = np.append(x_train, np.rot90(x_train, 2, axes=(1,2)), axis=0)
y_train = np.append(y_train, np.rot90(y_train, 2, axes=(1,2)), axis=0)
x_train = np.append(x_train, np.rot90(x_train, 1, axes=(1,2)), axis=0)
y_train = np.append(y_train, np.rot90(y_train, 1, axes=(1,2)), axis=0)

In [ ]:
%%time  
    
def transform_y(y, transform_base=None):
    y_flatten = []

    for k in range(len(y)):
        y_flatten += [y[k].flatten()]

    y = np.array(y_flatten)

    y_transformed = np.zeros(y.shape)
    if transform_base is None:
        transform_base = {}
        
    alpha = len(transform_base)

    for k in range(len(y)):
        for j in range(len(y[k])):
            if y[k,j] not in transform_base.keys():
                transform_base[y[k,j]] = alpha
                alpha += 1

            y_transformed[k, j] = transform_base[y[k,j]]
            
    return y_transformed, transform_base

def untransform_y(y, transform_base):
    transform_base_reverse = {}
    for key, value in transform_base.items():
        transform_base_reverse[value] = key
    
    y_untransformed = np.empty(y.shape, dtype='str')
    for k in range(len(y)):
        for j in range(len(y[k])):
            y_untransformed[k, j] = transform_base_reverse[y[k,j]]
    
    return y_untransformed
    
    

y_train_transformed, transform_base = transform_y(y_train)
y_valid_transformed, transform_base = transform_y(y_valid, transform_base=transform_base)

In [ ]:
y_train_transformed = tf.one_hot(y_train_transformed, len(transform_base)).numpy()
y_valid_transformed = tf.one_hot(y_valid_transformed, len(transform_base)).numpy()

In [ ]:
model = models.Sequential()
model.add(layers.ZeroPadding2D((1,1), input_shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.ZeroPadding2D((1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(25*6, activation='sigmoid'))
model.add(layers.Reshape((25, 6)))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['categorical_crossentropy', 'accuracy'])

history = model.fit(x_train, y_train_transformed, epochs=50, validation_data=(x_valid, y_valid_transformed))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Test on real photos

In [ ]:
x_test = []
y_test = []

with open('database_test.json') as json_file:
    y = json.load(json_file)

for file_name in os.listdir('data_test'):
    img_path = os.path.join('data_test', file_name)
    img = image.load_img(img_path, target_size=(32, 32))
    x = image.img_to_array(img)
    x = x/255
    
    """
    plt.figure(figsize=(3,3))
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x, cmap=plt.cm.binary)
    plt.show()
    """
    
    x_test += [x]
    y_test += [np.array(y[file_name])]
    
x_test = np.array(x_test)
y_test_transformed, transform_base = transform_y(y_test, transform_base=transform_base)

In [ ]:
y_test_transformed

In [ ]:
y_predict = np.argmax(model.predict(x_test), axis=2)

In [ ]:
f1_score(y_predict.flatten(),
         y_test_transformed.flatten(), average='weighted')

In [ ]:
random_y = copy.deepcopy(y_test_transformed.flatten())
random.shuffle(random_y)

In [ ]:
f1_score(random_y,
         y_test_transformed.flatten(), average='weighted')

In [ ]:
sample_image = image.array_to_img(x_test[1])
x = image.img_to_array(sample_image)
x = x/255
    
plt.figure(figsize=(3,3))
plt.xticks([])
plt.yticks([])
plt.grid(False)
plt.imshow(x, cmap=plt.cm.binary)
plt.show()

In [ ]:
sample_predict = np.argmax(model.predict(x_test), axis=2)[1].reshape(5,5)
untransform_y(sample_predict, transform_base)